In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing

2023-01-13 20:32:57.303605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-13 20:32:57.797261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:$LD_LIBRARY_PATH
2023-01-13 20:32:57.797317: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:$LD_LIBRARY_PATH
2023-01-13 20:32:57.797321: W tensorflow/compiler/tf

In [2]:
# Bypass issues with invoking notebook with server arguments
sys.argv.clear()
sys.argv.append("")

In [3]:
%run utils/attention_graph.py
%run utils/mlflow_query.py
%run utils/loading.py
%run utils/comparison.py
%run utils/ranks.py

In [ ]:
mlflow_helper = MlflowHelper(pkl_file=Path("mlflow_run_df.pkl"))
#mlflow_helper.query_runs(pkl_file=Path("mlflow_run_df.pkl"))

In [ ]:
sequences = pd.read_pickle("../data/sequences_df.pkl")
sequences

In [ ]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = Path("../data/logs_aggregated_concurrent.csv")
huawei_config.traces_root_directory = Path("../data/concurrent_data/traces/")
huawei_config.final_log_file = Path("../data/huawei.pkl")
huawei_config.remove_dates_from_payload = True

sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
#huawei_full_data = sequence_preprocessor.load_full_data()

In [ ]:
huawei_full_data.columns

# Manually step through program flow

In [ ]:
sequences_df = sequence_preprocessor.load_data()

In [ ]:
sequences_df.head()

In [ ]:
sequences_df["url_cluster_path"]

In [ ]:
# TF settings

import tensorflow as tf
import random

tensorflow_seed = 7796
random_seed = 82379498237

tf.random.set_seed(tensorflow_seed)
random.seed(random_seed)



# Collect sequence metadata

In [ ]:
from src.features import sequences
from src.features.sequences import transformer

sequence_column_name = sequence_preprocessor.sequence_column_name

transformer_config = sequences.SequenceConfig()
transformer_config.x_sequence_column_name = "fine_log_cluster_template"
transformer_config.y_sequence_column_name = "attributes"
transformer_config.predict_full_y_sequence_wide = True

transformer = transformer.NextPartialSequenceTransformerFromDataframe(transformer_config)

metadata = transformer.collect_metadata(sequences_df, sequence_column_name)

# Dataset exploration

In [4]:
dataset_df = pd.read_csv(Path("../data/logs_aggregated_concurrent.csv")).fillna("").astype(str).replace(np.nan, "", regex=True)

/tmp/ipykernel_842114/1652825543.py:1: DtypeWarning: Columns (23,24,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_df = pd.read_csv(Path("../data/logs_aggregated_concurrent.csv")).fillna("").astype(str).replace(np.nan, "", regex=True)


# Drain URL cluster

In [ ]:
log_only_data = sequence_preprocessor._load_log_only_data()

In [ ]:
log_only_data.columns

In [ ]:
log_only_data.describe()

In [ ]:
log_df = sequence_preprocessor._read_log_df()

In [ ]:
log_df.describe()

In [ ]:
log_df_url = sequence_preprocessor._add_url_drain_clusters(log_df)
log_df_url.describe()

In [5]:
from src.features.preprocessing import spell

In [6]:
parser = spell.LogParser(indir="../data/")

TypeError: expected string or bytes-like object

In [ ]:
parser.parse(logs_aggregated_concurrent.csv")